# Projet NLP

## Le contexte :

**Cdiscount** possède un catalogue qui comporte environ **80 Millions** de références **produits**.

A titre de comparaison un hypermarché peut posséder entre 20 000 et 80 000 références dans ses magasins.

Cdiscount se présente comme un **multi-spécialiste** et vend des produits organisés dans **25 grandes thématiques** aussi variées que la téléphonie, le bricolage, ou encore le vin.

C'est au final dans **plus de 8 000 catégories** que sont classés ses produits.

Plus de **90 %** de ces offres proviennent de sa **marketplace** de vendeurs tiers.

Et enfin tous les jours c'est **500 000** nouvelles références produits qui sont proposées à l'intégration dans son catalogue.

## La problématique : 

Du fait de la volumétrie quotidienne et surtout du fait que les vendeurs externes ne connaissent pas en détail les catégories disponibles, la catégorisation des nouveaux produits ne peut pas se faire manuellement.

Cdiscount doit donc **développer des algorithmes pour catégoriser** efficacement tout ces produits. 

Les algorithmes doivent être précis, une mauvaise catégorisation fait que le produit n'est pas présenté sur les bonnes pages, ne se vend pas, ce qui représente une perte sur le Chiffre d'Affaires.

Les algorithmes doivent aussi être rapides pour traiter toute la volumétrie.

Le but de ce projet est de développer un algorithme qui va se baser sur le contenu textuel des fiches produits pour identifier la catégorie à laquelle il appartient.

Vous devrez mettre en oeuvre les méthodes de NLP et de machine learning présentées en cours et TP pour générer vos prédictions.

## Les données : 

Vous aurrez accès dans ce projet aux fiches produits issues du catalogue produit de Cdiscount. Chaque ligne du dataset contient les 4 colonnes suivantes :
    
- Identifiant produit : idp
- L'intitulé produit : title
- La description : description
- Le prix : price
        
Vous avez à votre disposition 2 fichiers:

- Le fichier train.csv.gz
- Le fichier test.csv.gz

Le fichier train.csv.gz contient une colonne supplémentaire "category", qui contient la catégorie des produits. Dans ce fichier vous trouverez 500 fiches produits pour chacune des catégories du catalogue que vous aller traiter.

Il y a 600 catégories produits distinctes dans le jeu de donnée.

Ce fichier va vous servir à entrainer votre modèle.
    
- Les catégories produits se composent de 3 niveaux, du plus générique au plus précis, elles sont séparées par les caractères " > "
- Dans l'évaluation de la précision du modèle, cette hiérarchie aurra son importance
- La fonction d'évaluation de l'erreur est donnée un peu plus loin dans ce notebook 

In [1]:
import pandas

train = pandas.read_csv("train.csv.gz", nrows=5)
train


,idp,title,description,price,category
0,175565,"Arteza Feuilles Tatouages Temporaires, Pour le...",Pack de 10 feuilles : Contient plus de 100 tat...,24.99,HYGIENE - BEAUTE - PARFUM > MAQUILLAGE VISAGE ...
1,547892,Petit pot baleine pour l’apprentissage de la p...,Confortable : notre petit pot AnaBaby est ergo...,43.99,PUERICULTURE > TOILETTE BEBE > POT
2,42792,Les jeunes et la nature humaine : l'esprit de ...,Comment garantir l'avenir à une humanité plein...,19.50,LIBRAIRIE > SCIENCES HUMAINES > ETHNO - ANTHRO...
3,266447,Ht Montre Outil Ouvrir Boîtier Fond Vissé Clé ...,spécifications: et emballé jaw distance régla...,8.61,BIJOUX - LUNETTES - MONTRES > MONTRES > OUTILS...
4,1105868,Le générateur de fumée froide,8 10 heures de fumée propre et fraiche avec ...,15.00,ELECTROMENAGER > PETIT APPAREIL DE CUISSON > B...


Le fichier test.csv.gz lui ne contient pas la colonne "category", c'est à vous de développer un modèle de machine learning pour générer une prédiction de la catégorie associée à chaque produit.

Ce fichier contient 1 Million de produits.

In [2]:
test = pandas.read_csv("test.csv.gz", nrows=5)
test

,idp,title,description,price
0,5333572,Portable Usb rechargeable Neckband Lazy Neck H...,Portable USB rechargeable Neckband Lazy Neck H...,7.01
1,5425923,Coque Sony Xperia Z3 à motif en Silicone Gel P...,Coque Sony Xperia Z3 en TPU gel à motif Robert...,15.99
2,2123710,Fidget Spinner Tri Spinner Black Aluminum Meta...,123 123 123 123 123,49.99
3,579148,Taie Pour Oreiller Anatomique Temp Control 1558T,Réf. 1558T…Voir la présentation,14.00
4,3105464,Bracelet Women's Ss Fresh Water Cultured 6 7mm...,Bracelet te pièce étonnante vous permettra d...,113.99


## Des précisions sur la métrique que l'on souhaite optimiser 

On cherche bien évidement à catégoriser de manière la plus pertinente possible les produits.

Mais il va nous arriver de faire des erreurs, et ces erreurs ne vont pas être pénalisées de la même manière.

Chaque catégorie produit se décompose en 3 sous catégories qui s'écrivent sous le format "catégorie 1 > catégorie 2 > catégorie 3" :

- Il sera très pénalisant de se tromper sur la catégorie la plus générique (catégorie 1)
- Se tromper sur la catégorie la plus spécifique (catégorie 3) si les 2 premières catégories sont justes n'entrainera qu'une petite pénalité.
- Pour simuler aussi la réalité on considère qu'il y a 2 catégories 1 plus sensibles. Les catégories 'ADULTE - EROTIQUE' et 'VIN - ALCOOL - LIQUIDES' sont des catégories qui ne doivent pas être montrées à des mineurs. Classer par erreur ces produits dans une catégorie accessible a des mineurs est pénalisé encore plus fortement.  

La fonction exacte d'évaluation est définie par :

In [4]:
# Plus ce score est grand moins on est content
def error(real_category, predicted_category):
    # On a trouvé la bonne catégorie
    if real_category == predicted_category:
        return 0
    
    # On extrait les sous catégories
    real_categories = real_category.split(" > ")
    pred_categories = predicted_category.split(" > ")
    # On flag les catégories adultes
    adult_categories = ['ADULTE - EROTIQUE', 'VIN - ALCOOL - LIQUIDES']
    real_is_adult = real_categories[0] in adult_categories
    pred_is_adult = pred_categories[0] in adult_categories
    
    # Attention non symmétrie de l'erreur !
    if real_is_adult and not pred_is_adult:
        error = 10_000
        return error
    
    # On identifie à quel niveau on s'est trompé
    for real, pred, error in zip(real_categories, pred_categories, [100, 10, 1]):
        if real != pred:
            return error
    raise ValueError("Different categories, but no difference found")

## Challenge

A partir de maintenant et jusqu'à la soutenance à lieu un challenge entre vous. 

Vous devez soumettre vos prédictions sur les produits du fichier test.

Un script d'évaluation viendra calculer et mettre à jour les scores de votre équipe.

Le but est d'obtenir l'erreur totale la plus faible sur la métrique définie ci-dessus.

- Le notebook [Baseline](Baseline.ipynb) contient toutes les fonctions pour générer un fichier de prédiction dans le format attendu
- C'est aussi le TP du jour pour vous mettre le pied à l'étrier sur le challenge
- Ce notebook contient aussi les fonctions pour déposer le fichier de prédiction dans le répertoire /home/cisd-jacq/projet/evaluation/ qui est le répertoire de soumission
- Un job d'évaluation tourne toutes les 30min sur la plateforme de formation, il calcule les scores des fichiers déposés dans le dossier et met à jour le leaderboard
- Ce leaderboard est accessible à tous en exécutant le notebook [Leaderboard](Leaderboard.ipynb)


**Question Bonus** :

Un grand nombre des fiches produits du catalogue sont de faible qualité. 

L'une des raisons de cette mauvaise qualité vient du fait que certains vendeurs étrangers utilisent des outils de traduction automatique de faible qualité.

Pouvez-vous créer un modèle qui détecte ces fiches produits traduites automatiquement?

Attention c'est une question ouverte, je ne vous ai pas fourni les données d'entrainement dans le jeu de données. A ne traiter que si vous avez déjà terminé la 1ère partie du projet.

## Le jour de la soutenance

Avant de commencer votre soutenance je vous remettrai le fichier soutenance.csv.gz ayant un format identique à test.csv.gz et une volumétrie d'environ 100 000 lignes.
- Ce fichier contiendra de nouveaux produits pour lesquels vous devrez fournir des prédictions.
- Vous devez donc fournir le jour de la soutenance un script, ou un notebook, pouvant lire ce fichier et générer les prédictions.
- Ce script devra être capable de tourner dans les délais de votre présentation (< 20min)

Vous ferez une présentation d'environ 10 min, en utilisant des slides ou en présentant un notebook avec des annotations en markdown :
- Présentez la démarche que vous avez utilisé pour résoudre le problème
- Présentez les différentes techniques de traitement des données, les modèles de machine learning utilisés dans votre projet en justifiant vos choix
- Il peut être intéressant d'évoquer des pistes que vous avez exploré et qui n'ont pas marché
- Un focus sur les stratégies que vous avez mises en oeuvre pour prendre en compte la métrique spécifique du problème sera très apprécié
- Si vous dénichez des fun facts dans les données celà sera aussi apprécié
- Et enfin un bonus si vous apportez des éléments de réponse sur la question bonus

## Les critères de notation

4 points seront pris en compte pour la notation du projet : 
- Votre participation au challenge entre aujourd'hui et la soutenance
- Votre prestation sur la présentation le jour de la soutenance
- Votre capacité à fournir le jour de la soutenance un script/notebook qui génère des prédictions sur le fichier de validation fourni ce jour là
- La documentation/les commentaires du code/notebook que vous restituez